# Batch Split and Merge Excels

In [1]:
work_dir="course_datas/c15_excel_split_merge"
splits_dir=f"{work_dir}/splits"

import os
if not os.path.exists(splits_dir):
    os.mkdir(splits_dir)
    

In [2]:
import pandas as pd

In [3]:
df_source = pd.read_excel(f"{work_dir}/crazyant_blog_articles_source.xlsx")

In [4]:
df_source.head()

,id,title,tags
0,2585,Tensorflow怎样接收变长列表特征,"python,tensorflow,特征工程"
1,2583,Pandas实现数据的合并concat,"pandas,python,数据分析"
2,2574,Pandas的Index索引有什么用途？,"pandas,python,数据分析"
3,2564,机器学习常用数据集大全,"python,机器学习"
4,2561,一个数据科学家的修炼路径,数据分析


In [5]:
df_source.index

RangeIndex(start=0, stop=258, step=1)

In [6]:
df_source.shape

(258, 3)

In [7]:
total_row_count = df_source.shape[0]
total_row_count

258

## Batch Split Excels

In [8]:
# 这个大excel，会拆分给这几个人
user_names = ["xiao_shuai", "xiao_wang", "xiao_ming", 
              "xiao_lei", "xiao_bo", "xiao_hong"]

In [9]:
split_size = total_row_count // len(user_names)
if total_row_count % len(user_names) != 0:
    split_size += 1

split_size

43

In [10]:
seasons = ['Spring', 'Summer', 'Fall', 'Winter']
list(enumerate(seasons))

[(0, 'Spring'), (1, 'Summer'), (2, 'Fall'), (3, 'Winter')]

In [11]:
df_subs = []
for idx, user_name in enumerate(user_names):
    # iloc的开始索引
    begin = idx*split_size
    # iloc的结束索引
    end = begin+split_size
    # 实现df按照iloc拆分
    df_sub = df_source.iloc[begin:end]
    # 将每个子df存入列表
    df_subs.append((idx, user_name, df_sub))

In [12]:
for idx, user_name, df_sub in df_subs:
    file_name = f"{splits_dir}/crazyant_blog_articles_{idx}_{user_name}.xlsx"
    df_sub.to_excel(file_name, index=False)

## Batch merge Excels

In [13]:
import os
excel_names = []
for excel_name in os.listdir(splits_dir):
    excel_names.append(excel_name)
excel_names

['crazyant_blog_articles_1_xiao_wang.xlsx',
 'crazyant_blog_articles_0_xiao_shuai.xlsx',
 'crazyant_blog_articles_5_xiao_hong.xlsx',
 'crazyant_blog_articles_4_xiao_bo.xlsx',
 'crazyant_blog_articles_3_xiao_lei.xlsx',
 'crazyant_blog_articles_2_xiao_ming.xlsx']

In [14]:
df_list = []

for excel_name in excel_names:
    # 读取每个excel到df
    excel_path = f"{splits_dir}/{excel_name}"
    df_split = pd.read_excel(excel_path)
    # 得到username
    username = excel_name.replace("crazyant_blog_articles_", "").replace(".xlsx", "")[2:]
    print(excel_name, username)
    # 给每个df添加1列，即用户名字
    df_split["username"] = username
    
    df_list.append(df_split)

crazyant_blog_articles_1_xiao_wang.xlsx xiao_wang
crazyant_blog_articles_0_xiao_shuai.xlsx xiao_shuai
crazyant_blog_articles_5_xiao_hong.xlsx xiao_hong
crazyant_blog_articles_4_xiao_bo.xlsx xiao_bo
crazyant_blog_articles_3_xiao_lei.xlsx xiao_lei
crazyant_blog_articles_2_xiao_ming.xlsx xiao_ming


In [15]:
df_merged = pd.concat(df_list)

In [16]:
df_merged.shape

(258, 4)

In [17]:
df_merged.head()

,id,title,tags,username
0,2120,Zookeeper并不保证读取的是最新数据,zookeeper,xiao_wang
1,2089,Mybatis源码解读-初始化过程详解,mybatis,xiao_wang
2,2076,怎样借助Python爬虫给宝宝起个好名字,"python,爬虫",xiao_wang
3,2022,Mybatis源码解读-设计模式总结,"mybatis,设计模式",xiao_wang
4,2012,打工者心态、主人公意识、个人公司品牌,程序人生,xiao_wang


In [18]:
df_merged["username"].value_counts()

xiao_shuai    43
xiao_hong     43
xiao_ming     43
xiao_wang     43
xiao_bo       43
xiao_lei      43
Name: username, dtype: int64

In [19]:
df_merged.to_excel(f"{work_dir}/crazyant_blog_articles_merged.xlsx", index=False)